# Interactive RAG Demo 🔬
MongoDB + Voyage AI + Ollama

## Install Dependencies

In [2]:
%pip install python-dotenv voyageai pymongo ollama

Note: you may need to restart the kernel to use updated packages.


## Imports and Client Initialization

import os
import datetime
from dotenv import load_dotenv
import voyageai
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import ollama
import json
import time

# --- Load Environment Variables ---
# Make sure you have a .env file in the same directory as this notebook
load_dotenv()

VOYAGE_API_KEY = os.getenv("VOYAGE_API_KEY")
MONGO_URI = os.getenv("MONGO_URI")
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://localhost:11434")
DB_NAME = "rag_demo_db"
COLLECTION_NAME = "knowledge_base"

# --- Initialize Clients ---
try:
    # Voyage AI Client
    vo = voyageai.Client(api_key=VOYAGE_API_KEY)
    print("✅ Voyage AI client initialized.")

    # MongoDB Client
    mongo_client = MongoClient(MONGO_URI, server_api=ServerApi('1'))
    db = mongo_client[DB_NAME]
    collection = db[COLLECTION_NAME]
    mongo_client.admin.command('ping')
    print("✅ Successfully connected to MongoDB!")

    # Ollama Client
    ollama_client = ollama.Client(host=OLLAMA_HOST)
    print("✅ Ollama client initialized.")

except Exception as e:
    print(f"🔥 Error initializing clients: {e}")

# Helper to print JSON nicely
def print_json(data):
    print(json.dumps(data, indent=2))

## 1. Document Ingestion Pipeline
Here, we'll simulate uploading a document, chunking it, embedding it, and inserting it into the database.

### Step 1.1 - Load and Chunk Document

In [4]:
def chunk_text(text: str, chunk_size: int = 1024, overlap: int = 128) -> list[dict]:
    """Splits text into overlapping chunks."""
    if not text:
        return []
    chunks = []
    start = 0
    chunk_id = 1
    while start < len(text):
        end = start + chunk_size
        chunks.append({"chunk_id": chunk_id, "text": text[start:end]})
        start += chunk_size - overlap
        chunk_id += 1
    return chunks

In [10]:
document_name = "rich-dad-poor-dad.txt"
file_path = "./docs/" + document_name

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()
    print("File content successfully loaded into 'text_content' variable.")
    # You can now work with the 'text_content' variable
    # print(text_content[:200]) # Print the first 200 characters to verify
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

document_chunks = chunk_text(text_content)

print(f"📄 Document chunked into {len(document_chunks)} chunks.")
print("--- First Chunk ---")
print_json(document_chunks[0])

File content successfully loaded into 'text_content' variable.
📄 Document chunked into 7 chunks.
--- First Chunk ---
{
  "chunk_id": 1,
  "text": "\nRich Dad Poor Dad - Book Summary\nRich Dad Poor Dad, published in 1997 by authors Robert Kiyosaki and Sharon Lechter, has sold over 32 million copies in more than 51 languages. The book, which was endorsed by celebrities such as Oprah Winfrey, stayed on the New York Times best-sellers list for six years. This summary highlights six rules that the authors believe can help you make your money work for you.\n\nThe author, Robert Kiyosaki, had two dads: his biological \"Poor Dad\" and his friend Mike's \"Rich Dad\". Kiyosaki's Poor Dad worked hard his whole life, was highly educated, but had a different philosophy on money than his Rich Dad. The Rich Dad, who didn't have as many degrees, made much more money.\n\nRule 1: The rich do not work for money\nThe key takeaway of this rule is that the poor work for money, while the rich have money work

### Step 1.2 - Embed a Single Chunk

In [9]:
# We'll embed the first chunk as a demonstration to inspect the output
print("--- Running a single-chunk embedding example ---")
chunk_to_embed_example = document_chunks[0]['text']

try:
    embedding_response_example = vo.embed([chunk_to_embed_example], model="voyage-3-large", input_type="document")
    embedding_vector_example = embedding_response_example.embeddings[0]
    
    print("✅ Example chunk embedded successfully!")
    print(f"Vector dimensions: {len(embedding_vector_example)}")
    print("--- First 5 dimensions ---")
    print(embedding_vector_example[:5])
    
except Exception as e:
    print(f"🔥 Error embedding example chunk: {e}")

✅ Chunk embedded successfully!
Vector dimensions: 1024
--- First 5 dimensions ---
[-0.04736002907156944, 0.056065917015075684, -0.025943545624613762, 0.006529415957629681, 0.06128944829106331]


### Step 1.3 - Embed and Insert all Document Chunks

In [18]:
print(f"--- Starting bulk processing for {len(document_chunks)} chunks ---")
chunks_processed = 0

try:
    for chunk in document_chunks:
        if !(chunks_processed == 0) && (chunks_processed % 3 == 0):
            print("Waiting 60 seconds due to Voyage AI Free Tier limits ...")
            time.sleep(60)
        # 1. Embed the chunk
        embedding_response = vo.embed([chunk['text']], model="voyage-3-large", input_type="document")
        embedding_vector = embedding_response.embeddings[0]
        
        # 2. Prepare the document for insertion
        document_to_insert = {
            "text": chunk['text'],
            "embedding": embedding_vector,
            "source": document_name,
            "metadata": {
                "chunk_id": chunk['chunk_id'],
                "upload_date": datetime.datetime.now(datetime.timezone.utc),
            },
        }
        
        # 3. Insert the document
        collection.insert_one(document_to_insert)
        chunks_processed += 1
        print(f"Processed and inserted chunk {chunks_processed}/{len(document_chunks)}")

    print(f"\n✅ Successfully embedded and stored all {len(document_chunks)} chunks for the document.")

except Exception as e:
    print(f"\n🔥 An error occurred at chunk {chunks_processed + 1}: {e}")

--- Starting bulk processing for 7 chunks ---


KeyboardInterrupt: 

## 2. Query Processing Pipeline

### Step 2.1 - Embed the User Query

In [12]:
user_query = "How many copies did Rich Dad Poor Dad sell?"

try:
    # Using input_type="query" is crucial for retrieval quality
    query_embedding_response = vo.embed([user_query], model="voyage-3-large", input_type="query")
    
    # Extract the query vector
    query_vector = query_embedding_response.embeddings[0]
    
    print("✅ Query embedded successfully!")
    print(f"Vector dimensions: {len(query_vector)}")
    print("--- First 5 dimensions ---")
    print(query_vector[:5])

except Exception as e:
    print(f"🔥 Error embedding query: {e}")

✅ Query embedded successfully!
Vector dimensions: 1024
--- First 5 dimensions ---
[-0.018907172605395317, -0.01000967901200056, -0.03215230628848076, -0.049745071679353714, 0.027501342818140984]


### Step 2.2 - Retrieve Documents from MongoDB

In [19]:
# Construct the MongoDB Atlas Vector Search aggregation pipeline
pipeline = [
    {
        "$vectorSearch": {
            "index": "vector_index", # The name of your vector index
            "path": "embedding",
            "queryVector": query_vector,
            "numCandidates": 100,
            "limit": 3, # Retrieve top 3 most relevant chunks
        }
    },
    {
        "$project": {
            "_id": 0,
            "text": 1,
            "source": 1,
            "score": {"$meta": "vectorSearchScore"},
        }
    },
]

try:
    retrieved_docs = list(collection.aggregate(pipeline))
    print(f"✅ Retrieved {len(retrieved_docs)} documents from MongoDB.")
    print("--- Retrieved Documents ---")
    print_json(retrieved_docs)

except Exception as e:
    print(f"🔥 Error retrieving documents: {e}")

✅ Retrieved 3 documents from MongoDB.
--- Retrieved Documents ---
[
  {
    "text": "\nRich Dad Poor Dad - Book Summary\nRich Dad Poor Dad, published in 1997 by authors Robert Kiyosaki and Sharon Lechter, has sold over 32 million copies in more than 51 languages. The book, which was endorsed by celebrities such as Oprah Winfrey, stayed on the New York Times best-sellers list for six years. This summary highlights six rules that the authors believe can help you make your money work for you.\n\nThe author, Robert Kiyosaki, had two dads: his biological \"Poor Dad\" and his friend Mike's \"Rich Dad\". Kiyosaki's Poor Dad worked hard his whole life, was highly educated, but had a different philosophy on money than his Rich Dad. The Rich Dad, who didn't have as many degrees, made much more money.\n\nRule 1: The rich do not work for money\nThe key takeaway of this rule is that the poor work for money, while the rich have money work for them. Most people work for a monthly salary, but the fear

### Step 2.3 - Rerank Retrieved Documents

In [20]:
# We only need the text of the documents for reranking
docs_to_rerank = [doc["text"] for doc in retrieved_docs]

try:
    rerank_response = vo.rerank(
        query=user_query,
        documents=docs_to_rerank,
        model="rerank-2.5-lite",
        top_k=3,
    )
    
    # Re-order the original documents based on the reranker's response
    reranked_docs = [retrieved_docs[result.index] for result in rerank_response.results]

    print("✅ Documents reranked successfully.")
    print("--- Reranked Documents (in order of relevance) ---")
    print_json(reranked_docs)

except Exception as e:
    print(f"🔥 Error reranking documents: {e}")

✅ Documents reranked successfully.
--- Reranked Documents (in order of relevance) ---
[
  {
    "text": "\nRich Dad Poor Dad - Book Summary\nRich Dad Poor Dad, published in 1997 by authors Robert Kiyosaki and Sharon Lechter, has sold over 32 million copies in more than 51 languages. The book, which was endorsed by celebrities such as Oprah Winfrey, stayed on the New York Times best-sellers list for six years. This summary highlights six rules that the authors believe can help you make your money work for you.\n\nThe author, Robert Kiyosaki, had two dads: his biological \"Poor Dad\" and his friend Mike's \"Rich Dad\". Kiyosaki's Poor Dad worked hard his whole life, was highly educated, but had a different philosophy on money than his Rich Dad. The Rich Dad, who didn't have as many degrees, made much more money.\n\nRule 1: The rich do not work for money\nThe key takeaway of this rule is that the poor work for money, while the rich have money work for them. Most people work for a monthly 

### Step 2.4 - Construct LLM Prompt

In [21]:
# Use the reranked documents for the context
context_docs = reranked_docs 

context = ""
for i, doc in enumerate(context_docs):
    context += f"--- Document {i+1} (Source: {doc.get('source', 'N/A')}) ---\n"
    context += f"{doc.get('text', '')}\n\n"

prompt = f"""Use the following documents to answer the question. If the answer is not in the documents, say 'I cannot answer this question based on the provided documents.'

Documents:
{context}

Question: {user_query}

Answer:"""

print("✅ Final prompt constructed.")
print("--- LLM Prompt ---")
print(prompt)

✅ Final prompt constructed.
--- LLM Prompt ---
Use the following documents to answer the question. If the answer is not in the documents, say 'I cannot answer this question based on the provided documents.'

Documents:
--- Document 1 (Source: rich-dad-poor-dad.txt) ---

Rich Dad Poor Dad - Book Summary
Rich Dad Poor Dad, published in 1997 by authors Robert Kiyosaki and Sharon Lechter, has sold over 32 million copies in more than 51 languages. The book, which was endorsed by celebrities such as Oprah Winfrey, stayed on the New York Times best-sellers list for six years. This summary highlights six rules that the authors believe can help you make your money work for you.

The author, Robert Kiyosaki, had two dads: his biological "Poor Dad" and his friend Mike's "Rich Dad". Kiyosaki's Poor Dad worked hard his whole life, was highly educated, but had a different philosophy on money than his Rich Dad. The Rich Dad, who didn't have as many degrees, made much more money.

Rule 1: The rich do 

### Step 2.5 - Generate Final Answer with Ollama

In [27]:
try:
    final_response = ollama_client.generate(
        model='llama3',
        prompt=prompt,
    )
    
    print("✅ Final answer generated.")
    print("--- LLM Final Answer ---")
    print(final_response['response'])

except Exception as e:
    print(f"🔥 Error generating answer with Ollama: {e}")

✅ Final answer generated.
--- LLM Final Answer ---
According to Document 1, "Rich Dad Poor Dad" sold over 32 million copies in more than 51 languages.
